# dynoNet: Exercise
## Application to Ground Vibration Test of an F-16 aircraft. Description of the benchmark: https://www.nonlinearbenchmark.org/benchmarks/f-16-gvt

### Course on Deep Learning for System Identification
### Authors: Dario Piga, Marco Forgione
### Lugano, April 8th, 2024

Repeat the same exercise on the identification of the F16 aircraft.  **Use a dynoNet** instead of Feedforward and RNN networks.

Hyper-parameter to select:
* structure of the network 
* input and output lags of the LTI blocks
* structure of the Static Nonlinearities
* sub-sequence length for training and batch size
* number of initial samples skipped in the creation of the loss 
* optimizer's hyper-parameters (learning rate, maximum number of epochs)

Run the following cell to import possible useful packages and classes.

In [ ]:
import matplotlib.pyplot as plt
import os
import numpy as np
import dynonet
from dynonet.lti import MimoLinearDynamicalOperator
from dynonet.static import MimoStaticNonLinearity
from sklearn.preprocessing import StandardScaler
import pandas as pd
import torch 
import torch.nn as nn # the pytorch nn model contains all the layers we need to define a feedforward NN 
import torch.nn.functional as F
from torchsummary import summary
import torch.optim as optim
from sklearn.metrics import r2_score

# Enable interactive figures for Jupyter Notebooks.
%matplotlib widget 
from torch.utils.data import Dataset, DataLoader
from dataset import F16DS_seq

Since dynoNet receives as an input the input sequence of the system, the same *DataSet* class *F16DS_seq* used for training RNN can be also used in this exercise. Each element of the *F16DS_seq* Dataset is a pair with:
- input sub-sequences of a given length (since we are considering two inputs, we have 2 input sub-sequences)
- corresponding output sequences (if you consider all the three outputs, you will have 3 outputs)

For your convenience, you can run the cell below to upload the dataset and instantiate the *F16DS_seq* class.

In [ ]:
input_size = 2
seq_len = 10000 # length of subsequence
batch_size = 4
ind_out = [0, 1, 2] # it should be a list

folder = os.path.join('..', 'Datasets', 'F16')
f_train_ds =  os.path.join(folder, 'F16Data_SineSw_Level3.csv')
f_test_ds = os.path.join(folder, 'F16Data_SineSw_Level4_Validation.csv')


dict_ds = {'train': [], 'test': [],}
dict_ds['train'] = pd.read_csv(f_train_ds) 
dict_ds['test'] = pd.read_csv(f_test_ds)


ds_mean = dict_ds['train'].mean() 
ds_std = dict_ds['train'].std()
dict_ds['train']  = (dict_ds['train'] - ds_mean)/ds_std
dict_ds['test']  = (dict_ds['test'] - ds_mean)/ds_std

F16DS_train = F16DS_seq(pd_file=dict_ds['train'], seq_len = seq_len, ind_out=ind_out)
loader_train = DataLoader(F16DS_train, shuffle=True, batch_size=batch_size)
u, y = next(iter(loader_train))
print(y.shape)